## Using Unsloth to finetune

In [1]:

import os 
import sys 

from datasets import Dataset

from dotenv import find_dotenv, load_dotenv

load_dotenv()

/home/owen/anaconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

Leveraging Unsloth notebooks for finetuning

In [2]:
max_seq_length = 16000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


In [3]:
from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    # model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.6: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3080 Ti. Max memory: 11.753 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset


## Get dataset

In [6]:
dataset_finetune = load_dataset("CPSC532/arxiv_qa_data",
                                name="test_dataset_2024OCT23",
                                split="train",
                                token=os.getenv('HUGGINGFACE_API_KEY'),
                                )


In [7]:
dataset_finetune

Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 85
})

In [8]:

dataset_finetune['question'][0]

"What is the purpose of using prompt tuning in the framework described in the paper 'Visual prompt tuning'?"

In [9]:

dataset_finetune['answer'][0]

'According to the paper "Visual Prompt Tuning for Generative Transfer Learning" [1], the primary purpose of using prompt tuning in their framework is to adapt a pre-trained generative vision transformer model to a new target distribution or domain with minimal additional training data.\n\nPrompt tuning involves prepending learnable tokens called prompts to the input sequence of visual tokens, which guides the pre-trained transformer model to generate images that conform to the target distribution. The prompt parameters are learned via gradient descent while keeping the pre-trained transformer parameters frozen.\n\nThe authors argue that prompt tuning is a more efficient and effective way to adapt generative vision transformers to new domains compared to other transfer learning methods such as full fine-tuning or adapter tuning. They also claim that prompt tuning allows for better control over image generation, enabling the model to produce diverse and high-quality images.\n\nIn particu

Convert dataset to messages format

In [10]:
def convert_to_messages_format(example):
    return [
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['answer']},
    ]

In [11]:
dataset_finetune = dataset_finetune.map(
    lambda x: {
        'conversations' : convert_to_messages_format(x)
        }
)

In [12]:
dataset_finetune = dataset_finetune.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 85/85 [00:00<00:00, 5174.47 examples/s]


In [13]:
dataset_finetune['text'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the purpose of using prompt tuning in the framework described in the paper \'Visual prompt tuning\'?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAccording to the paper "Visual Prompt Tuning for Generative Transfer Learning" [1], the primary purpose of using prompt tuning in their framework is to adapt a pre-trained generative vision transformer model to a new target distribution or domain with minimal additional training data.\n\nPrompt tuning involves prepending learnable tokens called prompts to the input sequence of visual tokens, which guides the pre-trained transformer model to generate images that conform to the target distribution. The prompt parameters are learned via gradient descent while keeping the pre-trained transformer parameters frozen.\n\nThe authors argue that prom

## Dataset generated, now finetune the model

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_finetune,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,  # Affects memory usage
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2, # Affects memory usage
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 20, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map: 100%|██████████| 85/85 [00:00<00:00, 4195.88 examples/s]


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. Look into this

In [15]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map: 100%|██████████| 85/85 [00:00<00:00, 4981.85 examples/s]


In [16]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the purpose of using prompt tuning in the framework described in the paper \'Visual prompt tuning\'?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAccording to the paper "Visual Prompt Tuning for Generative Transfer Learning" [1], the primary purpose of using prompt tuning in their framework is to adapt a pre-trained generative vision transformer model to a new target distribution or domain with minimal additional training data.\n\nPrompt tuning involves prepending learnable tokens called prompts to the input sequence of visual tokens, which guides the pre-trained transformer model to generate images that conform to the target distribution. The prompt parameters are learned via gradient descent while keeping the pre-trained transformer parameters frozen.\n\nThe authors argue that prom

In [17]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                    \n\nThe authors of the paper "Visual Prompt Tuning for Generative Transfer Learning" are:\n\n1. Kihyuk Sohn\n2. Huiwen Chang\n3. Jos´e Lezama\n4. Luisa Polania\n5. Han Zhang\n6. Yuan Hao\n7. Irfan Essa\n8. Lu Jiang\n\nThey are affiliated with Google Research.<|eot_id|>'

In [18]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3080 Ti. Max memory = 11.753 GB.
3.275 GB of memory reserved.


In [19]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 85 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 194,510,848


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


  0%|          | 1/200 [00:02<09:50,  2.97s/it]

{'loss': 1.4582, 'grad_norm': 1.5018233060836792, 'learning_rate': 4e-05, 'epoch': 0.09}


  1%|          | 2/200 [00:05<09:18,  2.82s/it]

...

{'loss': 0.0012, 'grad_norm': 0.01932643912732601, 'learning_rate': 1.0256410256410257e-06, 'epoch': 18.51}


100%|██████████| 200/200 [10:05<00:00,  2.34s/it]

{'loss': 0.001, 'grad_norm': 0.01832760125398636, 'learning_rate': 0.0, 'epoch': 18.6}


100%|██████████| 200/200 [10:07<00:00,  3.04s/it]

{'train_runtime': 607.1811, 'train_samples_per_second': 2.8, 'train_steps_per_second': 0.329, 'train_loss': 0.13805641622602707, 'epoch': 18.6}


In [20]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

607.1811 seconds used for training.
10.12 minutes used for training.
Peak reserved memory = 10.723 GB.
Peak reserved memory for training = 7.448 GB.
Peak reserved memory % of max memory = 91.236 %.
Peak reserved memory for training % of max memory = 63.371 %.


## Run Inference

In [25]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def get_response(user_query):
    messages = [
    {"role": "user", "content": user_query},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                            temperature = 1.5, min_p = 0.1)
    return tokenizer.batch_decode(outputs)

In [26]:
dataset_finetune['question'][0]

"What is the purpose of using prompt tuning in the framework described in the paper 'Visual prompt tuning'?"

Need to investigate how changing the question affects responses

In [27]:
resp = get_response(dataset_finetune['question'][0])
print(resp[0].split("<|start_header_id|>assistant<|end_header_id|>")[1])



According to the paper "Visual Prompt Tuning for Generative Transfer Learning" [1], the primary purpose of using prompt tuning in their framework is to adapt a pre-trained generative vision transformer model to a new target distribution or domain with minimal additional training data.

Prompt tuning involves prepending learnable tokens called prompts to the


## Save to HF

In [28]:
model.push_to_hub_gguf(
        "CPSC532/finetuned_model", # Change hf to your username!
        tokenizer,
        
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = os.getenv("HUGGINGFACE_API_KEY"), # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 32.83 out of 62.67 RAM for saving.


100%|██████████| 28/28 [00:00<00:00, 90.82it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0', 'q5_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at CPSC532/finetuned_model into bf16 GGUF format.
The output location will be /home/owen/Desktop/github/532/implementation/finetuning/CPSC532/finetuned_model/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: finetuned_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
IN

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/CPSC532/finetuned_model
Unsloth: Uploading GGUF to Huggingface Hub...


...

Saved GGUF to https://huggingface.co/CPSC532/finetuned_model
Unsloth: Uploading GGUF to Huggingface Hub...


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/CPSC532/finetuned_model


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


Saved Ollama Modelfile to https://huggingface.co/CPSC532/finetuned_model
